In [1]:
#@formatter:off
%load_ext autoreload
%autoreload 2
#@formatter:on

In [31]:
!pip install datasets tensor2tensor tensorflow-text rouge_score transformers tqdm
!pip install -U tensorflow-text==2.9.0-rc1
!pip install -U tensorflow

In [4]:
import pickle
from src.LMGenerator import LMGenerator
from src.Metrics import get_metrics_for_all_prompts
from src.tokenize_utils import *
from src.SimpleTransformerLMHead import SimpleTransformerLMHead
from src.Config import Config
from tensorflow.python.keras.saving.save import load_model
from datasets import load_dataset
from pprint import pprint

# Load model from model.h5
load_lm_model = SimpleTransformerLMHead().load("model_save")
vocab = pickle.load(open('model_save/vocab.pkl', 'rb'))


# [SimpleTransformerLMHead] Initializing...
# [SimpleTransformerLMHead] Loading model from model_save...


---

In [12]:
references = list(tokenize(load_dataset("wikitext", "wikitext-2-raw-v1", split="test")['text']))


Reusing dataset wikitext (/root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


In [14]:
references[:5]


[['=', 'Robert', 'Boulter', '='],
 ['Robert',
  'Boulter',
  'is',
  'an',
  'English',
  'film',
  ',',
  'television',
  'and',
  'theatre',
  'actor'],
 ['He',
  'had',
  'a',
  'guest',
  '@-@',
  'starring',
  'role',
  'on',
  'the',
  'television',
  'series',
  'The',
  'Bill',
  'in',
  '2000'],
 ['This',
  'was',
  'followed',
  'by',
  'a',
  'starring',
  'role',
  'in',
  'the',
  'play',
  'Herons',
  'written',
  'by',
  'Simon',
  'Stephens',
  ',',
  'which',
  'was',
  'performed',
  'in',
  '2001',
  'at',
  'the',
  'Royal',
  'Court',
  'Theatre'],
 ['He',
  'had',
  'a',
  'guest',
  'role',
  'in',
  'the',
  'television',
  'series',
  'Judge',
  'John',
  'Deed',
  'in',
  '2002']]

In [38]:
import random
n_prompts = 5
def get_n_references(n_prompts,references,l=100):
  p_r = []
  for _ in range(n_prompts):
    i = random.randint(0, len(references)-l)
    rs = references[i:i+l]
    p = rs[0]
    p_r.append((' '.join(p), rs))
  return zip(*p_r)


In [44]:
from src.Metrics import Metrics, bleu, rouge, perplexity
import tqdm
def generate_all_sampling(model, prompt, n_tokens=50):
    print(prompt)
    greedy = model.generate(prompt, n_tokens, 'greedy')
    random = model.generate(prompt, n_tokens, 'random')
    top_k = model.generate(prompt, n_tokens, 'top_k')
    top_p = model.generate(prompt, n_tokens,'top_p')
    return greedy, random, top_k, top_p

def get_metrics(predictions, references):
    metric_values = {'predictions': predictions}

    # tokenize the input texts and get reference texts
    predictions = list(tokenize(predictions))
    predict_len = len(predictions)
    n_references = references[:predict_len]

    # get metrics
    metric_values.update(bleu.compute(predictions=predictions, references=list(to_bleu_references(n_references))))
    metric_values.update(rouge.compute(predictions=lword_to_lstr(predictions), references=lword_to_lstr(n_references)))
    metric_values.update(perplexity.compute(input_texts=lword_to_lstr(predictions), model_id='gpt2'))

    return metric_values

def get_metrics_for_all_sampling(generator, prompt, references, n_times=1):
    metrics = {
        'greedy': [],
        'random': [],
        'top_k': [],
        'top_p': []
    }    
    for _ in range(n_times):
      greedy, random, top_k, top_p = generate_all_sampling(generator, prompt)
      metrics['greedy'].append(get_metrics(greedy, references))
      metrics['random'].append(get_metrics(random, references))
      metrics['top_k'].append(get_metrics(top_k, references))
      metrics['top_p'].append(get_metrics(top_p, references))
    return metrics

def get_metrics_for_all_prompts(generator, prompts, references, n_times=1):
    metrics = {}
    for i, prompt in tqdm.tqdm(enumerate(prompts)):
        metrics[i] = get_metrics_for_all_sampling(generator, prompt, references, n_times)
    return metrics

In [45]:

references = list(tokenize(load_dataset("wikitext", "wikitext-2-raw-v1", split="test")['text']))
prompts, prompt_refrences = get_n_references(n_prompts, references)
metrics = get_metrics_for_all_prompts(load_lm_model, prompts, references, n_times=10)

Reusing dataset wikitext (/root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
0it [00:00, ?it/s]

This difference generally results in JTWC maximum winds appearing higher than the maximum winds described by the JMA for the same cyclone
# [SimpleTransformerLMHead] Generating...
Generated 51 tokens
# [SimpleTransformerLMHead] Generating...
Generated 51 tokens
# [SimpleTransformerLMHead] Generating...
Generated 51 tokens
# [SimpleTransformerLMHead] Generating...
Generated 51 tokens


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

...

Using pad_token, but it is not set yet.


  0%|          | 0/25 [00:00<?, ?it/s]

Using pad_token, but it is not set yet.


  0%|          | 0/25 [00:00<?, ?it/s]

5it [31:47, 381.46s/it]


In [46]:
pprint(metrics)

Streaming output truncated to the last 5000 lines.
                                 168.00148010253906,
                                 123.41194152832031,
                                 168.00148010253906,
                                 115.0157699584961,
                                 133.01162719726562,
                                 95.4771499633789,
                                 222.8732147216797,
                                 129.71514892578125,
                                 163.53013610839844,
                                 222.8732147216797,
                                 202.60543823242188,
                                 168.00148010253906,
                                 95.4771499633789,
                                 149.00682067871094,
                                 129.71514892578125,
                                 129.04942321777344,
                                 95.4771499633789,
                                 149.00682067871094,
    

In [ ]:
pickle.dump(metrics, open('metrics.pkl', 'wb'))